# Гладкий силует акули з shapefile через криві Безьє 5-го порядку

In [ ]:
import fiona
import shapely.geometry
import numpy as np
import matplotlib.pyplot as plt

# Зчитування шейпфайлу
shp_path = "shark_shapefile/shark/shark_contour.shp"
with fiona.open(shp_path, 'r') as shapefile:
    shape = shapely.geometry.shape(next(iter(shapefile))["geometry"])
    coords = np.array(shape.exterior.coords)

# Центрування та нормалізація
centered_coords = coords - coords.mean(axis=0)
scale = max(np.ptp(centered_coords[:, 0]), np.ptp(centered_coords[:, 1]))
normalized_coords = centered_coords / scale * 20

# Візуалізація
plt.figure(figsize=(7, 7))
plt.plot(normalized_coords[:, 0], -normalized_coords[:, 1], color='blue')
plt.title("Контур акули з shapefile")
plt.axis('equal')
plt.grid(True)
plt.show()


In [ ]:
from Bezier import Bezier

def chunk_closed(points, chunk_size=6):
    total = len(points)
    segments = []
    for i in range(0, total, chunk_size):
        if i + chunk_size <= total:
            segments.append(points[i:i+chunk_size])
        else:
            remaining = points[i:]
            needed = chunk_size - len(remaining)
            segment = np.vstack([remaining, points[:needed]])
            segments.append(segment)
    return segments

# Побудова кривих
segments = chunk_closed(normalized_coords, 6)
t_vals = np.linspace(0, 1, 300)

fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.grid(True, linestyle='--', alpha=0.3)
ax.set_title("Гладкий силует акули з shapefile (Bezier-5)")

for idx, seg in enumerate(segments):
    curve = Bezier.Curve(t_vals, seg)
    ax.plot(curve[:, 0], -curve[:, 1], color='blue')
    midpoint = Bezier.Point(0.5, seg)
    ax.text(midpoint[0], -midpoint[1], f"S{idx+1}", fontsize=8, color='darkred')

plt.show()
